# Unit 9: LightFM

You almost made it - this is the final lesson and it is also going to be the easiest one.

As you may already assume - there are a lot of recommender packages in Python out there. In this lesson we will look at LightFM - an easy to use and lightweight implementation of different approaches and algorithms (FM, BPR, WARP, ...) to perform CF, CBF and hybrid recommenders.

Within a few lines of code we set-up, train and use a recommender for recommendations.

* [LightFM on GitHub](https://github.com/lyst/lightfm)
* [LightFM documentation](https://making.lyst.com/lightfm/docs/home.html)

In [240]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix

from recsys_training.data import Dataset, genres

In [4]:
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k
from lightfm import LightFM

/Users/mkurovski/anaconda3/envs/recsys_training/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [7]:
ml100k_ratings_filepath = '../../data/raw/ml-100k/u.data'
ml100k_item_filepath = '../../data/raw/ml-100k/u.item'
ml100k_user_filepath = '../../data/raw/ml-100k/u.user'

## Load Data

### You may easily load Movielens Data ...

In [306]:
data = fetch_movielens(min_rating=4.0, genre_features=True)

In [307]:
data

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 49906 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 5469 stored elements in COOrdinate format>,
 'item_features': <1682x1701 sparse matrix of type '<class 'numpy.float32'>'
 	with 4575 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'genre:Thriller', 'genre:War', 'genre:Western'], dtype=object),
 'item_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object)}

### But, we want to use the exact same data and split that we used in the lessons before

In [321]:
data = Dataset(ml100k_ratings_filepath)
data.filter(min_rating=4.0)
data.rating_split(seed=42)

#### Transform our training and testing data into sparse matrices

In [322]:
# Train DataFrame to Train COO Matrix
ratings = data.train_ratings["rating"].values
# We subtract 1 to make user/item ids 0-index-based
rows = data.train_ratings["user"].values - 1
cols = data.train_ratings["item"].values - 1

train_mat = coo_matrix((ratings, (rows, cols)),
                       shape=(data.n_users, data.n_items))


# Test DataFrame to Test COO Matrix
ratings = data.test_ratings["rating"].values
# We subtract 1 to make user/item ids 0-index-based
rows = data.test_ratings["user"].values - 1
cols = data.test_ratings["item"].values - 1

test_mat = coo_matrix((ratings, (rows, cols)),
                      shape=(data.n_users, data.n_items))

In [323]:
train_mat

<943x1682 sparse matrix of type '<class 'numpy.int64'>'
	with 44300 stored elements in COOrdinate format>

In [324]:
test_mat

<943x1682 sparse matrix of type '<class 'numpy.int64'>'
	with 11075 stored elements in COOrdinate format>

## Collaborative Filtering

In [433]:
params = {
    'no_components': 10,
    'loss': 'bpr',
    'learning_rate': 0.07,
    'random_state': 42,
    'user_alpha': 0.0002,
    'item_alpha': 0.0002
}

epochs = 10

N = 10

In [434]:
cf_model = LightFM(**params)

In [435]:
cf_model.fit(train_mat, epochs=epochs, verbose=True)

Epoch: 100%|██████████| 10/10 [00:00<00:00, 48.66it/s]


### Evaluate the `MAP@10` on test data

If we provide training data with evaluation, known positives will be removed.

In [436]:
prec_at_N = precision_at_k(cf_model, test_mat, train_mat, k=N)

In [437]:
prec_at_N.mean()

0.17415851

### Evaluate the `MAP@10` on train data

In [438]:
prec_at_N = precision_at_k(cf_model, train_mat, k=N)

In [439]:
prec_at_N.mean()

0.4393843

Maybe try adding some regularization to improve the recommendation relevancy - simply add `user_alpha` and `item_alpha` to the `params` dictionary and find appropriate values.

## Hybrid (CF + CBF)

### Load user and item features

In [440]:
def min_max_scale(val, bounds):
    min_max_range = bounds['max']-bounds['min']
    return (val-bounds['min'])/min_max_range


def user_profiler(group):
    genre_dist = group[genres].mean()
    year_dist = group['release_year'].describe()[['mean', 'std', '50%']]

    return pd.concat((genre_dist, year_dist), axis=0)


def get_user_profiles(ratings: pd.DataFrame,
                      item_feat: pd.DataFrame,
                      min_rating: float = 4.0) -> pd.DataFrame:
    ratings = ratings[ratings.rating >= min_rating]
    ratings = ratings[['user', 'item']]
    ratings = ratings.merge(item_feat, on='item', how='left')
    ratings.drop(['item'], axis=1, inplace=True)

    grouped = ratings.groupby('user')
    profiles = grouped.apply(user_profiler).reset_index()
    profiles.rename(columns={'50%': 'median'}, inplace=True)
    
    return profiles


item_feat = pd.read_csv(ml100k_item_filepath, sep='|', header=None,
                        names=['item', 'title', 'release', 'video_release', 'imdb_url']+genres,
                        engine='python')

user_feat = pd.read_csv(ml100k_user_filepath, sep='|', header=None,
                        names=['user', 'age', 'gender', 'occupation', 'zip'])

# Infer the release year
idxs = item_feat[item_feat['release'].notnull()].index
item_feat.loc[idxs, 'release_year'] = item_feat.loc[idxs, 'release'].str.split('-')
item_feat.loc[idxs, 'release_year'] = item_feat.loc[idxs, 'release_year'].apply(lambda val: val[2]).astype(int)

# Impute median release year value for the items with missing release year
top_year = item_feat.loc[idxs, 'release_year'].astype(int).describe()['50%']
idx = item_feat[item_feat['release'].isnull()].index
item_feat.loc[idx, 'release_year'] = top_year

# Min-max scale the release year
item_year_bounds = {'min': item_feat['release_year'].min(),
                    'max': item_feat['release_year'].max()}
item_feat['release_year'] = item_feat['release_year'].apply(
    lambda year: min_max_scale(year, item_year_bounds))

# Drop other columns
item_feat.drop(['title', 'release', 'video_release', 'imdb_url'], axis=1, inplace=True)

# Min-max scale the age
user_age_bounds = {'min': user_feat['age'].min(),
                   'max': user_feat['age'].max()}
user_feat['age'] = user_feat['age'].apply(lambda age: min_max_scale(age, user_age_bounds))

# Transform gender characters to numerical values (categories)
genders = sorted(user_feat['gender'].unique())
user_gender_map = dict(zip(genders, range(len(genders))))
user_feat['gender'] = user_feat['gender'].map(user_gender_map)

# Transform occupation strings to numerical values (categories)
occupations = sorted(user_feat['occupation'].unique())
user_occupation_map = dict(zip(occupations, range(len(occupations))))
user_feat['occupation'] = user_feat['occupation'].map(user_occupation_map)

# Transform the zip codes to categories keeping the first three digits and impute for missing
idxs = user_feat[~user_feat['zip'].str.isnumeric()].index
user_feat.loc[idxs, 'zip'] = '00000'
zip_digits_to_cut = 3
user_feat['zip'] = user_feat['zip'].apply(lambda val: int(val) // 10 ** zip_digits_to_cut)


profiles = get_user_profiles(data.train_ratings, item_feat)
user_feat = user_feat.merge(profiles, on='user', how='left')

occupation_1H = pd.get_dummies(user_feat['occupation'], prefix='occupation')
zip_1H = pd.get_dummies(user_feat['zip'], prefix='zip')

user_feat.drop(['occupation', 'zip', ], axis=1, inplace=True)
user_feat = pd.concat([user_feat, occupation_1H, zip_1H], axis=1)

user_feat.fillna(0, inplace=True)


user_feat.index = user_feat['user'].values
user_feat.drop('user', axis=1, inplace=True)

item_feat.index = item_feat['item'].values
item_feat.drop('item', axis=1, inplace=True)

In [441]:
(user_feat==0).sum().sum()/user_feat.size

0.8608033813918158

In [442]:
(item_feat==0).sum().sum()/item_feat.size

0.8640309155766944

In [443]:
# Create User Feature COO Matrix
# user_feat_mat = coo_matrix(np.eye(data.n_users))
user_feat_mat = coo_matrix(np.concatenate((user_feat.values, np.eye(data.n_users)), axis=1))

# Create Item Feature COO Matrix
# item_feat_mat = coo_matrix(np.eye(data.n_items))
item_feat_mat = coo_matrix(np.concatenate((item_feat.values, np.eye(data.n_items)), axis=1))

In [444]:
user_feat_mat

<943x1084 sparse matrix of type '<class 'numpy.float64'>'
	with 19451 stored elements in COOrdinate format>

In [445]:
item_feat_mat

<1682x1702 sparse matrix of type '<class 'numpy.float64'>'
	with 6256 stored elements in COOrdinate format>

### Model Training

In [454]:
params = {
    'no_components': 10,
    'loss': 'warp',
    'learning_rate': 0.07,
    'random_state': 42,
    'user_alpha': 0.0002,
    'item_alpha': 0.0002
}

epochs = 10

N = 10

In [455]:
hybrid_model = LightFM(**params)

hybrid_model.fit(train_mat,
                 user_features=user_feat_mat,
                 item_features=item_feat_mat,
                 epochs=epochs,
                 verbose=True)

Epoch: 100%|██████████| 10/10 [00:00<00:00, 19.44it/s]


In [456]:
prec_at_N = precision_at_k(hybrid_model,
                           test_mat,
                           train_mat,
                           k=N,
                           user_features=user_feat_mat,
                           item_features=item_feat_mat)

In [457]:
prec_at_N.mean()

0.19381107